In [3]:
import pandas as pd
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from sklearn.naive_bayes import MultinomialNB
from sklearn.multiclass import OneVsRestClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score
from pandas.plotting import scatter_matrix
from sklearn.neighbors import KNeighborsClassifier
from sklearn import metrics
import re

In [2]:
from tika import parser 
df = pd.DataFrame(columns = ['CandidateID', 'resume', 'cleaned_resume'])
for dirname, _, filenames in os.walk('./testResumes'):
    for filename in filenames:
        path = os.path.join(dirname, filename)
        raw = parser.from_file(path)
        df = df.append({'CandidateID' : filename[:-4], 'resume' : " ".join(raw['content'].strip().split('\n')[1:]), 'cleaned_resume' : ""}, 
                ignore_index = True)
df

,CandidateID,resume,cleaned_resume
0,candidate_004,O L I V I A S A N T O S CONSULTANT ANALYST ...,
1,candidate_005,ARMIN FITZGERALD D A T A M A N A G ...,
2,candidate_014,GRACE BAILRY M A C H I N E L E A R N I N G ...,
3,candidate_016,A M E L I A M A R T I N DATA SCIENCE MAJOR ...,
4,candidate_017,P R O F I L E I am a Junior Machine Learnin...,
5,candidate_018,EVELYNN ADAMS G R A D U A T E F R E S H E ...,
6,candidate_019,JOSHUA WHITE A S S I S T A N T S O F T W A ...,
7,candidate_028,"ASI Technologies Intern, Dec 2020 - Ongoin...",
8,candidate_029,Eli Scott F R E SHER Executive Profile See...,
9,candidate_031,P R O F I L E An avid explorer of analytics...,


In [4]:
def cleanResume(resumeText):
    resumeText = re.sub('http\S+\s*', ' ', resumeText)  # remove URLs
    resumeText = re.sub('RT|cc', ' ', resumeText)  # remove RT and cc
    resumeText = re.sub('#\S+', '', resumeText)  # remove hashtags
    resumeText = re.sub('@\S+', '  ', resumeText)  # remove mentions
    resumeText = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', resumeText)  # remove punctuations
    resumeText = re.sub(r'[^\x00-\x7f]',r' ', resumeText) 
    resumeText = re.sub('\s+', ' ', resumeText)  # remove extra whitespace
    return resumeText

In [5]:
df['cleaned_resume'] = df.resume.apply(lambda x: cleanResume(x))
print (df['cleaned_resume'][2])

 GRACE BAILRY M A C H I N E L E A R N I N G D E V E L O P M E N T A N D D E P L O Y M E N T SKILLS Software Engineer Developer Software Development Project Management Machine Learning Computer Vision OpenCV Python C Ruby Learning Algorithm EDUCATION B Tech in Electronics Computers from DVR College of Engineering Technology in 2017 Post Graduation in Software Development from CDAC Noida in 2019 OTHER ACTIVITIES Scalable Projects and their Deployment SDLC Pioneers Deployment of End to End ML Projects KVCC 20 Workshop PROJECTS Clustering of images based on feature management Creating a recommender system to understand false alarms on support side of real time services PERSONAL PROFILE I am a software engineer who has worked on several computer vision projects and their end to end deployment I am looking to join as a Machine Learning developer to closely work on problems and increase my grasp over the domain WORK BACKGROUND Eximius Design Pvt ltd company Software Development and Deployment

In [6]:
train = pd.read_csv("test.csv")
train.set_index('CandidateID',inplace=True)
df.set_index("CandidateID",inplace=True)

In [7]:
train = train.join(df)
train

,resume,cleaned_resume
CandidateID,,
candidate_014,GRACE BAILRY M A C H I N E L E A R N I N G ...,GRACE BAILRY M A C H I N E L E A R N I N G D ...
candidate_098,"VoyaGenius Labs Software Developer (ML), De...",VoyaGenius Labs Software Developer ML Dec 201...
candidate_075,KEIRON PAVARD D A T A E N G I N E E R SKIL...,KEIRON PAVARD D A T A E N G I N E E R SKILLS ...
candidate_016,A M E L I A M A R T I N DATA SCIENCE MAJOR ...,A M E L I A M A R T I N DATA SCIENCE MAJOR WO...
candidate_131,ZACHARY PEREZ P R O F I L E Good knowledge ...,ZACHARY PEREZ P R O F I L E Good knowledge of...
candidate_056,Christian Von Kelin J U N I O R A N A L Y S...,Christian Von Kelin J U N I O R A N A L Y S T...
candidate_141,P R O F I L E Software developer with a nic...,P R O F I L E Software developer with a niche...
candidate_044,L E A H A T K I N JUNIOR DEVELOPER WORK EX...,L E A H A T K I N JUNIOR DEVELOPER WORK EXPER...
candidate_029,Eli Scott F R E SHER Executive Profile See...,Eli Scott F R E SHER Executive Profile Seekin...


In [9]:
import nltk
from nltk.corpus import stopwords
import string
from wordcloud import WordCloud

oneSetOfStopWords = set(stopwords.words('english')+['``',"''"])
totalWords =[]
Sentences = train['resume'].values
cleanedSentences = ""
for i in range(0,60):
    cleanedText = cleanResume(Sentences[i])
    cleanedSentences += cleanedText
    requiredWords = nltk.word_tokenize(cleanedText)
    for word in requiredWords:
        if word not in oneSetOfStopWords and word not in string.punctuation:
            totalWords.append(word)
    
wordfreqdist = nltk.FreqDist(totalWords)
mostcommon = wordfreqdist.most_common(50)
print(mostcommon)
wc = WordCloud().generate(cleanedSentences)

[('E', 290), ('I', 243), ('A', 198), ('S', 161), ('R', 154), ('Data', 148), ('T', 145), ('N', 141), ('Learning', 125), ('C', 108), ('Machine', 108), ('L', 98), ('O', 94), ('D', 80), ('2019', 80), ('Python', 76), ('P', 71), ('data', 70), ('B', 63), ('Analysis', 57), ('2020', 55), ('using', 48), ('Tech', 43), ('learning', 40), ('U', 40), ('F', 35), ('Software', 33), ('SKILLS', 32), ('G', 31), ('based', 30), ('V', 28), ('SQL', 28), ('M', 27), ('Deep', 27), ('PROFILE', 26), ('WORK', 26), ('Science', 26), ('Analyst', 26), ('Processing', 25), ('Analytics', 25), ('J', 25), ('Engineer', 24), ('2018', 22), ('Natural', 22), ('Business', 22), ('H', 21), ('Developer', 21), ('PROJECTS', 21), ('machine', 21), ('Artificial', 21)]


In [10]:
train.drop(columns=['resume'],inplace=True)

In [15]:
train.to_csv('cleaned_test.csv')